<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/572_SEv2_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tests for Sales Enablement Orchestrator report generation.

In [ ]:
"""Tests for Sales Enablement Orchestrator report generation."""

import tempfile
from pathlib import Path

from config import SalesEnablementOrchestratorConfig
from agents.sales_enablement.orchestrator import create_sales_enablement_graph


def test_report_is_written_and_contains_sections():
    """Run full graph; assert report file exists and contains Pipeline, Rep, Top Priority sections."""
    with tempfile.TemporaryDirectory() as tmp:
        config = SalesEnablementOrchestratorConfig(reports_dir=tmp)
        graph = create_sales_enablement_graph(config=config)
        initial = {"lead_id": None, "rep_id": None, "focus_area": None, "errors": []}

        report_path = None
        for event in graph.stream(initial):
            for node_name, node_out in event.items():
                if node_name == "reporting":
                    report_path = node_out.get("report_file_path")
                    assert "enablement_report" in node_out
                    assert node_out.get("enablement_report"), "enablement_report should be non-empty"

        assert report_path, "reporting node should set report_file_path"
        path = Path(report_path)
        assert path.exists(), f"Report file should exist: {report_path}"

        content = path.read_text(encoding="utf-8")
        assert "Pipeline Summary" in content, "Report should include Pipeline Summary"
        assert "Rep Performance" in content, "Report should include Rep Performance"
        assert "Top Priority Leads" in content, "Report should include Top Priority Leads"
        assert "Sales Enablement Report" in content, "Report should have title"

#test results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_023_SalesEnablementOrchestrator % pytest test_sales_enablement_report.py -v
============================================================================= test session starts =============================================================================
platform darwin -- Python 3.13.7, pytest-9.0.2, pluggy-1.6.0 -- /Users/micahshull/Documents/AI_AGENTS/AI_AGENTS_023_SalesEnablementOrchestrator/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/micahshull/Documents/AI_AGENTS/AI_AGENTS_023_SalesEnablementOrchestrator
plugins: anyio-4.12.1, asyncio-1.3.0, langsmith-0.6.6, cov-7.0.0
asyncio: mode=Mode.STRICT, debug=False, asyncio_default_fixture_loop_scope=None, asyncio_default_test_loop_scope=function
collected 1 item

test_sales_enablement_report.py::test_report_is_written_and_contains_sections PASSED                                                                                    [100%]

============================================================================== 1 passed in 0.18s ==============================================================================


#Unit tests for Sales Enablement prioritization utility.

In [ ]:
"""Unit tests for Sales Enablement prioritization utility."""

from agents.sales_enablement.orchestrator.utilities.prioritization import build_prioritized_leads


def test_build_prioritized_leads_returns_sorted_by_score_desc():
    """Prioritized list should be sorted by priority_score descending."""
    leads = [
        {"lead_id": "L-A", "intent_score": 0.5, "budget_range": "25k-50k"},
        {"lead_id": "L-B", "intent_score": 0.9, "budget_range": "250k+"},
        {"lead_id": "L-C", "intent_score": 0.7, "budget_range": "50k-100k"},
    ]
    signals_lookup = {
        "L-A": {"engagement_score": 0.4, "deal_risk_score": 0.6, "urgency": "low"},
        "L-B": {"engagement_score": 0.9, "deal_risk_score": 0.1, "urgency": "high"},
        "L-C": {"engagement_score": 0.7, "deal_risk_score": 0.3, "urgency": "medium"},
    }
    deals_lookup = {}
    weights = {"intent_score": 0.3, "engagement_score": 0.25, "deal_risk_score": 0.2, "budget_range": 0.15, "urgency": 0.1}

    prioritized, top = build_prioritized_leads(leads, signals_lookup, deals_lookup, weights, top_n=10)

    scores = [p["priority_score"] for p in prioritized]
    assert scores == sorted(scores, reverse=True), "prioritized should be sorted by score desc"
    assert len(prioritized) == 3
    assert top[0]["lead_id"] == "L-B", "highest intent+engagement+urgency should rank first"
    assert top[0]["priority_score"] >= top[1]["priority_score"] >= top[2]["priority_score"]


def test_build_prioritized_leads_top_n_truncates():
    """top_priority_leads should have at most top_n items."""
    leads = [
        {"lead_id": "L-1", "intent_score": 0.8, "budget_range": "50k-100k"},
        {"lead_id": "L-2", "intent_score": 0.7, "budget_range": "50k-100k"},
        {"lead_id": "L-3", "intent_score": 0.6, "budget_range": "50k-100k"},
    ]
    signals_lookup = {f"L-{i}": {"engagement_score": 0.5, "deal_risk_score": 0.3, "urgency": "medium"} for i in (1, 2, 3)}
    deals_lookup = {}
    weights = {"intent_score": 0.3, "engagement_score": 0.25, "deal_risk_score": 0.2, "budget_range": 0.15, "urgency": 0.1}

    prioritized, top = build_prioritized_leads(leads, signals_lookup, deals_lookup, weights, top_n=2)

    assert len(prioritized) == 3
    assert len(top) == 2
    assert top[0]["lead_id"] == "L-1" and top[1]["lead_id"] == "L-2"


def test_build_prioritized_leads_required_keys():
    """Each prioritized item should have lead_id, priority_score, urgency, recommended_action, rationale."""
    leads = [{"lead_id": "L-X", "intent_score": 0.6, "budget_range": "25k-50k"}]
    signals_lookup = {"L-X": {"engagement_score": 0.5, "deal_risk_score": 0.4, "urgency": "high", "recommended_action": "schedule demo"}}
    deals_lookup = {}
    weights = {"intent_score": 0.3, "engagement_score": 0.25, "deal_risk_score": 0.2, "budget_range": 0.15, "urgency": 0.1}

    prioritized, top = build_prioritized_leads(leads, signals_lookup, deals_lookup, weights, top_n=5)

    for item in prioritized:
        assert "lead_id" in item and item["lead_id"] == "L-X"
        assert "priority_score" in item and 0 <= item["priority_score"] <= 100
        assert "urgency" in item
        assert "recommended_action" in item
        assert "rationale" in item and len(item["rationale"]) > 0


def test_build_prioritized_leads_skips_leads_without_lead_id():
    """Leads missing lead_id should be skipped."""
    leads = [{"intent_score": 0.9}]
    signals_lookup = {}
    deals_lookup = {}
    weights = {"intent_score": 0.3, "engagement_score": 0.25, "deal_risk_score": 0.2, "budget_range": 0.15, "urgency": 0.1}

    prioritized, top = build_prioritized_leads(leads, signals_lookup, deals_lookup, weights, top_n=5)

    assert len(prioritized) == 0
    assert len(top) == 0


# Unit tests for Sales Enablement reporting utilities

In [ ]:
"""Unit tests for Sales Enablement reporting utilities."""

from agents.sales_enablement.orchestrator.utilities.reporting import (
    build_enablement_report_md,
    build_pipeline_summary,
    build_rep_performance_summary,
)


def test_build_pipeline_summary_counts():
    """Pipeline summary should reflect active/won/lost counts and values."""
    deals = [
        {"status": "active", "deal_value_usd": 100_000, "probability": 0.5, "days_in_stage": 5, "risk_flags": []},
        {"status": "active", "deal_value_usd": 50_000, "probability": 0.8, "days_in_stage": 25, "risk_flags": ["pricing"]},
        {"status": "won", "deal_value_usd": 75_000},
        {"status": "lost", "deal_value_usd": 30_000},
    ]
    out = build_pipeline_summary(deals, thresholds={"stalled_deal_days_in_stage": 21})

    assert out["total_deals"] == 4
    assert out["active_deals"] == 2
    assert out["won_deals"] == 1
    assert out["lost_deals"] == 1
    assert out["total_pipeline_value"] == 150_000
    assert out["weighted_pipeline_value"] == 100_000 * 0.5 + 50_000 * 0.8
    assert out["win_rate"] == 0.5
    assert out["stalled_deals_count"] == 1
    assert out["at_risk_deals_count"] == 1


def test_build_pipeline_summary_empty_deals():
    """Empty deals should yield zero counts and zero values."""
    out = build_pipeline_summary([], None)
    assert out["total_deals"] == 0
    assert out["active_deals"] == 0
    assert out["total_pipeline_value"] == 0
    assert out["win_rate"] == 0
    assert out["stalled_deals_count"] == 0


def test_build_rep_performance_summary_structure():
    """Rep summary should have rep_id, active_deals, pipeline_value, quota_achievement, needs_coaching, top_opportunities."""
    deals = [
        {"rep_id": "SR-01", "status": "active", "deal_value_usd": 100_000, "probability": 0.6},
        {"rep_id": "SR-01", "status": "active", "deal_value_usd": 50_000, "probability": 0.4},
    ]
    reps = [
        {"rep_id": "SR-01", "quota_usd": 500_000, "year_to_date_revenue_usd": 400_000, "close_rate": 0.35},
    ]
    out = build_rep_performance_summary(deals, reps)

    assert len(out) == 1
    r = out[0]
    assert r["rep_id"] == "SR-01"
    assert r["active_deals"] == 2
    assert r["pipeline_value"] == 150_000
    assert r["quota_achievement"] == 0.8
    assert r["needs_coaching"] is False
    assert "top_opportunities" in r and len(r["top_opportunities"]) <= 3


def test_build_rep_performance_summary_needs_coaching():
    """needs_coaching should be True when quota_achievement < 0.7."""
    deals = []
    reps = [{"rep_id": "SR-X", "quota_usd": 1_000_000, "year_to_date_revenue_usd": 500_000, "close_rate": 0.2}]
    out = build_rep_performance_summary(deals, reps)
    assert out[0]["quota_achievement"] == 0.5
    assert out[0]["needs_coaching"] is True


def test_build_enablement_report_md_contains_sections():
    """Generated markdown should include Pipeline Summary, Rep Performance, Top Priority Leads."""
    ps = {"total_deals": 5, "active_deals": 3, "won_deals": 1, "lost_deals": 1, "total_pipeline_value": 200_000,
          "weighted_pipeline_value": 120_000, "average_deal_size": 66_666, "win_rate": 0.5,
          "stalled_deals_count": 1, "at_risk_deals_count": 0}
    rps = [{"rep_id": "SR-01", "active_deals": 2, "pipeline_value": 150_000, "close_rate": 0.3,
            "quota_achievement": 0.8, "needs_coaching": False, "top_opportunities": ["D-01", "D-02"]}]
    top = [{"lead_id": "L-01", "priority_score": 85, "urgency": "high", "recommended_action": "schedule demo"}]

    md = build_enablement_report_md(ps, rps, top)

    assert "# Sales Enablement Report" in md
    assert "## Pipeline Summary" in md
    assert "## Rep Performance" in md
    assert "## Top Priority Leads" in md
    assert "L-01" in md
    assert "85" in md
    assert "schedule demo" in md or "schedule" in md


In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_023_SalesEnablementOrchestrator % pytest test_sales_enablement_prioritization.py
============================================================================= test session starts =============================================================================
platform darwin -- Python 3.13.7, pytest-9.0.2, pluggy-1.6.0
rootdir: /Users/micahshull/Documents/AI_AGENTS/AI_AGENTS_023_SalesEnablementOrchestrator
plugins: anyio-4.12.1, asyncio-1.3.0, langsmith-0.6.6, cov-7.0.0
asyncio: mode=Mode.STRICT, debug=False, asyncio_default_fixture_loop_scope=None, asyncio_default_test_loop_scope=function
collected 4 items

test_sales_enablement_prioritization.py ....                                                                                                                            [100%]

============================================================================== 4 passed in 0.14s ==============================================================================


# Tests for Sales Enablement data loading

In [ ]:
"""Tests for Sales Enablement data loading (uses agents/data/ when run from project root)."""

from pathlib import Path

from agents.sales_enablement.orchestrator.utilities.data_loading import load_all_sales_data


def test_load_all_sales_data_returns_core_keys():
    """load_all_sales_data should return leads, sales_reps, interactions, deals, signals and lookups."""
    # Assume run from project root; agents/data lives under project root
    project_root = Path(__file__).resolve().parent
    data_dir = project_root / "agents" / "data"

    if not data_dir.exists():
        return  # skip if data dir not present (e.g. in a minimal checkout)

    data = load_all_sales_data(data_dir=data_dir)

    assert "leads" in data
    assert "sales_reps" in data
    assert "interactions" in data
    assert "deals" in data
    assert "signals" in data
    assert "leads_lookup" in data
    assert "reps_lookup" in data
    assert "signals_lookup" in data
    assert "interactions_lookup" in data
    assert "deals_lookup" in data


def test_load_all_sales_data_has_records():
    """With real agents/data/, we should get non-empty leads and deals."""
    project_root = Path(__file__).resolve().parent
    data_dir = project_root / "agents" / "data"

    if not data_dir.exists():
        return

    data = load_all_sales_data(data_dir=data_dir)

    assert len(data["leads"]) > 0
    assert len(data["deals"]) > 0
    assert len(data["leads_lookup"]) == len(data["leads"])
    assert len(data["signals_lookup"]) == len(data["signals"])


def test_load_all_sales_data_mvp2_when_present():
    """When thresholds/objections/content_assets exist, they should appear in the result."""
    project_root = Path(__file__).resolve().parent
    data_dir = project_root / "agents" / "data"

    if not data_dir.exists():
        return

    data = load_all_sales_data(data_dir=data_dir)

    if (data_dir / "thresholds.json").exists():
        assert "thresholds" in data and data["thresholds"] is not None
    if (data_dir / "objections.json").exists():
        assert "objections" in data and isinstance(data["objections"], list)
    if (data_dir / "content_assets.json").exists():
        assert "content_assets" in data and isinstance(data["content_assets"], list)
